In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
dir = '/content/drive/MyDrive/ARIA'
sys.path.append(dir)

import os
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install matplotlib==3.7.0
%pip install numpy==1.26.4

In [ ]:
import numpy as np
import jax
from jax import numpy as jnp
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle

import cv2
import os

dx, dy = 0.5, 0.5
ny, nx = 26, 49
dt = 0.4

with open(f'{dir}/dataset/dataset_v2.pickle', 'rb') as handle:
  dataset = pickle.load(handle)

train_set = list(range(len(dataset['INFLOW_LOCS'])))

INFLOW_LOCS = dataset['INFLOW_LOCS'][train_set,]
FLOW_TIMES = [jnp.array(dataset['FLOW_TIMES'][i]) for i in train_set]
SMOKE_FIELD = [jnp.array(dataset['SMOKE_FIELD'][i]) for i in train_set]
VELOCITY = dataset['VELOCITY']
REL_LOC = dataset['rel_loc']
TERRAIN = dataset['TERRAIN']
SPLINE_TCK = [dataset['SPLINE_TCK'][i] for i in train_set]

train_set = [0, 2, 4, 5, 6, 7, 8, 10, 12, 13, 15, 17, 19, 21, 23, 11, 14, 16, 25, 37, 38, 41, 42, 43, 49, 50]
train_subset = [8, 10, 12, 16, 19, 23, 25, 38]
# train_set = [6, 12, 8, 21,  2,  7, 15, 17, 10, 19, 23, 13]

for set_idx in range(len(INFLOW_LOCS)): #len(INFLOW_LOCS)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_smoke.pickle', 'rb') as bunch:
        smoke = pickle.load(bunch)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_advection.pickle', 'rb') as bunch:
        advection_terms = pickle.load(bunch)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_diffusion.pickle', 'rb') as bunch:
        diffusion_terms = pickle.load(bunch)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_closure.pickle', 'rb') as bunch:
        closure_terms = pickle.load(bunch)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_relerror.pickle', 'rb') as bunch:
        error_terms = pickle.load(bunch)
    with open(f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_inflow.pickle', 'rb') as bunch:
        inflow_terms = pickle.load(bunch)


    # Output video parameters
    if set_idx in train_subset:
        output_filename = f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_train.mp4'
    elif set_idx in train_set:
        output_filename = f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_trainfull.mp4'
    else:
        output_filename = f'{dir}/videos/22072025_{INFLOW_LOCS[set_idx]}_test.mp4'
    frame_width, frame_height = 960, 240
    fps = 25

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    # Generate frames
    frame_index = 1
    for i in range(len(smoke)):
        fig, axs = plt.subplots(2, 4, figsize=(20, 5))

        # Plot each array in its respective subplot
        temp = smoke[i]
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im1 = axs[0,0].imshow(temp, origin='lower')
        axs[0,0].set_title('Smoke')
        fig.colorbar(im1, ax=axs[0,0],fraction=0.046, pad=0.04)

        temp = advection_terms[i]
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im2 = axs[0,1].imshow(temp, origin='lower')
        axs[0,1].set_title('Advection')
        fig.colorbar(im2, ax=axs[0,1],fraction=0.046, pad=0.04)

        temp = diffusion_terms[i]
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im3 = axs[1,0].imshow(temp, origin='lower')
        axs[1,0].set_title('Diffusion')
        fig.colorbar(im3, ax=axs[1,0],fraction=0.046, pad=0.04)

        temp = closure_terms[i].squeeze()
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im4 = axs[1,1].imshow(temp, origin='lower')
        axs[1,1].set_title('Closure')
        fig.colorbar(im4, ax=axs[1,1],fraction=0.046, pad=0.04)

        temp = error_terms[i]
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im5 = axs[0,2].imshow(temp, origin='lower')
        axs[0,2].set_title('Relative Error')
        fig.colorbar(im5, ax=axs[0,2],fraction=0.046, pad=0.04)

        temp = inflow_terms[i]
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im6 = axs[1,2].imshow(temp, origin='lower')
        axs[1,2].set_title('Inflow')
        fig.colorbar(im6, ax=axs[1,2],fraction=0.046, pad=0.04)

        temp = jnp.log10(jnp.maximum(smoke[i],1e-12))
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im7 = axs[0,3].imshow(temp, vmin=-8, vmax=-4, origin='lower')
        axs[0,3].set_title(f'Smoke, t={"%.2f" % (i*dt)} (log scale)')
        fig.colorbar(im7, ax=axs[0,3],fraction=0.046, pad=0.04)

        # next time
        temp = np.log10(SMOKE_FIELD[set_idx][frame_index]+1e-12)
        temp = jnp.where(TERRAIN==1.0, temp, jnp.nan)
        im8 = axs[1,3].imshow(temp, vmin=-8, vmax=-4, origin='lower')
        axs[1,3].set_title(f'Desired Smoke, t={"%.2f" %FLOW_TIMES[set_idx][frame_index]} (log scale)')
        fig.colorbar(im8, ax=axs[1,3],fraction=0.046, pad=0.04)

        if (i+1)*dt>FLOW_TIMES[set_idx][frame_index]:
            frame_index+=1

        # Remove axes for cleaner look
        for ax in axs.flatten():
            ax.axis('off')

        # Save the figure to a temporary file
        # plt.tight_layout()
        # plt.savefig('frame.png')
        # plt.close(fig)
        fig.canvas.draw()
        img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frame = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        plt.close('all')
        # Read the image and resize it
        # frame = cv2.imread('frame.png')
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Write the frame to the video
        out.write(frame)

    # Clean up
    out.release()
    plt.close('all')

    print("Video generated successfully:", output_filename)

/tmp/ipython-input-3177728821.py:129: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')


Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[13  3]_trainfull.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[6 8]_test.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[ 6 15]_trainfull.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[13 20]_test.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[23 21]_trainfull.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[10 31]_trainfull.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[13 40]_trainfull.mp4
Video generated successfully: /content/drive/MyDrive/ARIA/videos/22072025_26C8_set1/videos/22072025_[13 45]_trainfull.mp4
Video generated successfully: /conte